# Demo 4: Deploying our MLFlow Model in Azure Machine Learning

**Authored by:** Joshua Isanan

**Date:** 07/28/2024

Let's load our MLClient with the necessary credentials needed.

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

import numpy as np

resource_group = "RESOURCE GROUP"
subscription_id = "SUBSCRIPTION ID"
workspace = "WORKSPACE NAME"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

Fetch our baseline model.

In [3]:
baseline_model = ml_client.models.get(name="baseline_model", version="2")

Create the endpoint name.

In [4]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

import datetime

endpoint_name = "purchase-prediction-" + datetime.datetime.now().strftime("%m%d")

In [5]:
endpoint_name

'purchase-prediction-0810'

Using the endpointname, let's create a ManagedOnlineEndpoint entity and configure the authentication mode for our endpoint.

(This may take a while. You may check its status on the Endpoints section in the Azure Machine Learning workspace)

In [6]:
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint that uses the Purchase Prediction ML Model.",
    auth_mode="key",
)

ml_client.begin_create_or_update(endpoint)

We then need to create a deployment that will host our ML Model. 

instance_type and instance_count can be modified to your liking if you need a resource that has more computing power.

In [7]:
endpoint_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=baseline_model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)

Let's then provision our resource and host our model there. 

(This may take a while. You may check its status on the Endpoints section in the Azure Machine Learning workspace)

In [8]:
ml_client.online_deployments.begin_create_or_update(endpoint_deployment)

Check: endpoint purchase-prediction-0810 exists


.......................................

We must then move all our traffic to our deployed resource.

In [9]:
endpoint.traffic = {"blue": 100}

ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://purchase-prediction-0730.southeastasia.inference.ml.azure.com/score', 'openapi_uri': 'https://purchase-prediction-0730.southeastasia.inference.ml.azure.com/swagger.json', 'name': 'purchase-prediction-0730', 'description': 'Endpoint that uses the Purchase Prediction ML Model.', 'tags': {}, 'properties': {'createdBy': 'Japhet Aguilar', 'createdAt': '2024-07-30T18:44:30.615237+0000', 'lastModifiedAt': '2024-07-30T18:44:30.615237+0000', 'azureml.onlineendpointid': '/subscriptions/b0194562-e534-46c1-998a-6e224b1e0a1b/resourcegroups/jisanan-mlops/providers/microsoft.machinelearningservices/workspaces/azure-ml-jisanan/onlineendpoints/purchase-prediction-0730', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/b0194562-e534-46c1-998a-6e224b1e0a1b/providers/Microsoft.MachineLearningServices/locations/southeastasia/mfeOperationsStatus/oeidp:9221e095-8aee-4a5c-

After that, we can now test our deployed endpoint. You can use the data below as a sample input.

In [ ]:
{"input_data": 
    {
        "columns": [
            "Age",
            "Gender",
            "AnnualIncome",
            "NumberOfPurchases",
            "ProductCategory",
            "TimeSpentOnWebsite",
            "LoyaltyProgram",
            "DiscountsAvailed"
        ],
        "data": [
            [1,2,3,4,1,6,7,8],
            [10,2,9,8,0,6,5,4]
        ],
        "index": [0,1]
    }
}